# Langchain Intro

In [1]:
# Built-in library
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pathlib import Path
import pandas as pd
import polars as pl
from pprint import pprint
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain.llms import OpenAI

### Using A Basic OpenAI Langchain LLM

- Without using a chain.
- Very simplistic.

<br>

[![image.png](https://i.postimg.cc/cLyVwqrs/image.png)](https://postimg.cc/Mv5PJF74)


In [3]:
_ = load_dotenv(find_dotenv())  # read local .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_GPT_MODEL: str = "gpt-3.5-turbo-instruct"
TEMPERATURE: float = 0.5

In [4]:
# Create LLM
llm: OpenAI = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    model=OPENAI_GPT_MODEL,
    temperature=TEMPERATURE,
)
prompt: str = "Write a very short poem."
result = llm(prompt=prompt)

print(result)

Love's sweet embrace,
Fills my heart with grace.
Forever entwined,
Our souls combined.

## Updated Logic

- Add Chains (from langchain)

<br>

[![image.png](https://i.postimg.cc/RVLp2WgH/image.png)](https://postimg.cc/0brCwyRk)

In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### LLM Chain

[![image.png](https://i.postimg.cc/05VBzdbB/image.png)](https://postimg.cc/0rJf3SGY)

In [6]:
# Used for text completion LLMs
prompt_template: str = """Write a very short {language} function that will {task}"""
code_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "task"], template=prompt_template
)
code_chain: LLMChain = LLMChain(llm=llm, prompt=code_prompt)

# Hardcoded inputs!
language, task = ("Python", "generate a list of numbers")

result = code_chain(inputs={"language": language, "task": task})
print(result)

{
    'language': 'Python',
    'task': 'generate a list of numbers',
    'text': '\n\ndef generate_list(start, stop, step):\n    return list(range(start, stop, step))'
}

In [7]:
print(result.get("text"))

def generate_list(start, stop, step):
    return list(range(start, stop, step))

## Connecting Chains Together

- Chains in series (sequential chain)

<br>

[![image.png](https://i.postimg.cc/8Cdgm9wV/image.png)](https://postimg.cc/GBpfcMZM)

In [8]:
from langchain.chains import SequentialChain

# Used for text completion LLMs
code_prompt_template: str = (
    """Write a very short {language} function that will {task}"""
)
unittest_prompt_template: str = (
    """Write a unit test for the following {language} code: \n{code}"""
)

# Propmt(s)
code_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "task"], template=code_prompt_template
)
unittest_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "code"], template=unittest_prompt_template
)

# Chain(s)
code_chain: LLMChain = LLMChain(
    llm=llm,
    prompt=code_prompt,
    output_key="code",  # from text to code
)
unittest_chain: LLMChain = LLMChain(
    llm=llm,
    prompt=unittest_prompt,
    output_key="test",  # from text to test
)
# Feed the output of code_chain into unittest_chain
final_chain: SequentialChain = SequentialChain(
    chains=[code_chain, unittest_chain],
    input_variables=["language", "task"],
    output_variables=["code", "test"],
)
# Hardcoded inputs!
language, task = ("Python", "generate a list of numbers")

result = final_chain(inputs={"language": language, "task": task})
print(result)

{
    'language': 'Python',
    'task': 'generate a list of numbers',
    'code': '\n\ndef generate_numbers(n):\n    return list(range(1, n+1))',
    'test': "\n\nimport unittest\n\nclass TestGenerateNumbers(unittest.TestCase):\n    \n    def 
test_generate_numbers(self):\n        self.assertEqual(generate_numbers(5), [1,2,3,4,5])\n        
self.assertEqual(generate_numbers(0), [])\n        self.assertEqual(generate_numbers(10), [1,2,3,4,5,6,7,8,9,10])\n
self.assertEqual(generate_numbers(-5), [])\n        self.assertEqual(generate_numbers(1), [1])\n\nif __name__ == 
'__main__':\n    unittest.main()"
}

In [9]:
# Output of the unittest_chain (key="test")
print(result.get("test"))

import unittest

class TestGenerateNumbers(unittest.TestCase):
    
    def test_generate_numbers(self):
        self.assertEqual(generate_numbers(5), [1,2,3,4,5])
        self.assertEqual(generate_numbers(0), [])
        self.assertEqual(generate_numbers(10), [1,2,3,4,5,6,7,8,9,10])
        self.assertEqual(generate_numbers(-5), [])
        self.assertEqual(generate_numbers(1), [1])

if __name__ == '__main__':
    unittest.main()

<hr><br>

# Simple Terminal Chatbot

<br>

### Chat Model

- It's optimized for conversations.
- It has the following inputs:
  - **System Message:** A system message is a message generated by the system or platform hosting the conversation. It is not authored by a human user or the AI model.
  
  - **Human Message:** Any message created by a human user that is directed towards the LLM for processing or interaction.

  - **AI/Assistant Message:** The output generated by the LLM in response to a human message or internal processing.
  
[![image.png](https://i.postimg.cc/FRgxXmJ6/image.png)](https://postimg.cc/YG0FQ5RN)

<br>

### ChatPrompt Template

[![image.png](https://i.postimg.cc/s2nTM32Z/image.png)](https://postimg.cc/4K96MkVJ)

In [10]:
OPENAI_CHAT_MODEL: str = "gpt-3.5-turbo"

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import (
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
)

In [12]:
chat_llm = ChatOpenAI(
    model=OPENAI_CHAT_MODEL,
    temperature=TEMPERATURE,
)

# Used for chat completion LLMs
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[HumanMessagePromptTemplate.from_template("{content}")],
)

chain: LLMChain = LLMChain(llm=chat_llm, prompt=prompt)

In [13]:
content: str = "Hello, who are you?"

result: dict[str, Any] = chain({"content": content})
print(result)

{
    'content': 'Hello, who are you?',
    'text': 'Hello! I am an AI assistant created by OpenAI. I am here to help answer any questions you may have. 
How can I assist you today?'
}

### Add Memory

- Memory allows LLMs to access and consider previous conversations or inputs, leading to more coherent and contextually relevant responses.

<br>

[![image.png](https://i.postimg.cc/MGPkWySn/image.png)](https://postimg.cc/8F6X4fsG)

<br><br>

#### Message PlaceHolder

- It's used to track the conversations.
- It contains the contents of the chat. i.e. the human and the AI messages.

[![image.png](https://i.postimg.cc/3wPBCRjL/image.png)](https://postimg.cc/p9JzPP68)


In [14]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

In [15]:
chat_llm = ChatOpenAI(
    model=OPENAI_CHAT_MODEL,
    temperature=TEMPERATURE,
)

memory = ConversationBufferMemory(
    memory_key="messages",  # key for storing the chat history
    return_messages=True,  # returns the promp classes. e.g. HumanMsg, AIMsg, etc
)

# Used for chat completion LLMs
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        HumanMessagePromptTemplate.from_template("{content}"),
        MessagesPlaceholder(
            variable_name="messages"
        ),  # used to track the conversations
    ],
)

chain: LLMChain = LLMChain(
    llm=chat_llm,
    prompt=prompt,
    memory=memory,  # Add memory (new!)
)

In [16]:
content: str = "Hello, who are you?"

result: dict[str, Any] = chain({"content": content})
print(result)

{
    'content': 'Hello, who are you?',
    'messages': [
        HumanMessage(content='Hello, who are you?', additional_kwargs={}, example=False),
        AIMessage(
            content='Hello! I am an AI language model developed by OpenAI. I am here to assist and engage in 
conversation with you. How can I help you today?',
            additional_kwargs={},
            example=False
        )
    ],
    'text': 'Hello! I am an AI language model developed by OpenAI. I am here to assist and engage in conversation 
with you. How can I help you today?'
}

In [17]:
# Follow up question
content: str = "What is NLP?"

result: dict[str, Any] = chain({"content": content})
print(result)

{
    'content': 'What is NLP?',
    'messages': [
        HumanMessage(content='Hello, who are you?', additional_kwargs={}, example=False),
        AIMessage(
            content='Hello! I am an AI language model developed by OpenAI. I am here to assist and engage in 
conversation with you. How can I help you today?',
            additional_kwargs={},
            example=False
        ),
        HumanMessage(content='What is NLP?', additional_kwargs={}, example=False),
        AIMessage(
            content='NLP stands for Natural Language Processing. It is a subfield of artificial intelligence (AI) 
that focuses on the interaction between computers and human language. NLP involves the development of algorithms 
and models that allow computers to understand, interpret, and generate human language in a way that is meaningful 
and useful. It encompasses tasks such as text classification, sentiment analysis, machine translation, question 
answering, and more. NLP has a wide range of applications, including virtual assistants, chatbots, language 
translation, text analysis, and information retrieval.',
            additional_kwargs={},
            example=False
        )
    ],
    'text': 'NLP stands for Natural Language Processing. It is a subfield of artificial intelligence (AI) that 
focuses on the interaction between computers and human language. NLP involves the development of algorithms and 
models that allow computers to understand, interpret, and generate human language in a way that is meaningful and 
useful. It encompasses tasks such as text classification, sentiment analysis, machine translation, question 
answering, and more. NLP has a wide range of applications, including virtual assistants, chatbots, language 
translation, text analysis, and information retrieval.'
}

In [18]:
# Follow up question
content: str = "How can I study it?"

result: dict[str, Any] = chain({"content": content})
print(result)

{
    'content': 'How can I study it?',
    'messages': [
        HumanMessage(content='Hello, who are you?', additional_kwargs={}, example=False),
        AIMessage(
            content='Hello! I am an AI language model developed by OpenAI. I am here to assist and engage in 
conversation with you. How can I help you today?',
            additional_kwargs={},
            example=False
        ),
        HumanMessage(content='What is NLP?', additional_kwargs={}, example=False),
        AIMessage(
            content='NLP stands for Natural Language Processing. It is a subfield of artificial intelligence (AI) 
that focuses on the interaction between computers and human language. NLP involves the development of algorithms 
and models that allow computers to understand, interpret, and generate human language in a way that is meaningful 
and useful. It encompasses tasks such as text classification, sentiment analysis, machine translation, question 
answering, and more. NLP has a wide range of applications, including virtual assistants, chatbots, language 
translation, text analysis, and information retrieval.',
            additional_kwargs={},
            example=False
        ),
        HumanMessage(content='How can I study it?', additional_kwargs={}, example=False),
        AIMessage(
            content='To study NLP, here are some steps you can follow:\n\n1. Learn the basics: Start by 
understanding the fundamental concepts and techniques in NLP. This includes topics like tokenization, 
part-of-speech tagging, named entity recognition, syntactic parsing, and semantic analysis.\n\n2. Programming 
skills: Develop your programming skills, as NLP often requires coding. Python is a popular language for NLP tasks, 
so learning Python and libraries like NLTK (Natural Language Toolkit) and spaCy can be beneficial.\n\n3. Read books
and research papers: There are several books and research papers available on NLP that can provide in-depth 
knowledge on various topics. Some recommended books include "Speech and Language Processing" by Jurafsky and 
Martin, "Foundations of Statistical Natural Language Processing" by Manning and Schütze, and "Natural Language 
Processing with Python" by Bird, Klein, and Loper.\n\n4. Online courses: Enroll in online courses that cover NLP 
concepts and techniques. Platforms like Coursera, Udemy, and edX offer courses on NLP, such as "Natural Language 
Processing" by Stanford University and "Deep Learning Specialization" by deeplearning.ai.\n\n5. Practice with 
datasets and projects: Apply your knowledge by working on NLP projects and experimenting with different datasets. 
This hands-on experience will help solidify your understanding and improve your skills.\n\n6. Stay updated: Stay 
informed about the latest advancements and research in NLP. Follow blogs, research papers, and conferences like ACL
(Association for Computational Linguistics) and EMNLP (Empirical Methods in Natural Language Processing) to keep up
with the latest trends.\n\n7. Join communities: Engage with the NLP community by participating in forums, 
discussion groups, and online communities. This allows you to connect with like-minded individuals, ask questions, 
and learn from others\' experiences.\n\nRemember, NLP is a vast and evolving field, so continuous learning and 
practice are key to mastering it.',
            additional_kwargs={},
            example=False
        )
    ],
    'text': 'To study NLP, here are some steps you can follow:\n\n1. Learn the basics: Start by understanding the 
fundamental concepts and techniques in NLP. This includes topics like tokenization, part-of-speech tagging, named 
entity recognition, syntactic parsing, and semantic analysis.\n\n2. Programming skills: Develop your programming 
skills, as NLP often requires coding. Python is a popular language for NLP tasks, so learning Python and libraries 
like NLTK (Natural Language Toolkit) and spaCy can be beneficial.\n\n3. Read books and research papers: Th

### Saving And Extending Conversations

- Add the ability for the chatbot to remember previous conversations.
    
<br>

[![image.png](https://i.postimg.cc/NGTgxztb/image.png)](https://postimg.cc/Dmvkv58b)

<br>

> We can use:
- a JSON file.

[![image.png](https://i.postimg.cc/B6FGMXrt/image.png)](https://postimg.cc/bDqMvNNj)

In [19]:
from langchain.memory import FileChatMessageHistory

In [20]:
chat_llm = ChatOpenAI(
    model=OPENAI_CHAT_MODEL,
    temperature=TEMPERATURE,
)

memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory(file_path="chat_history.json"),  # New!
    memory_key="messages",  # key for storing the chat history
    return_messages=True,  # returns the promp classes. e.g. HumanMsg, AIMsg, etc
)

# Used for chat completion LLMs
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        HumanMessagePromptTemplate.from_template("{content}"),
        MessagesPlaceholder(
            variable_name="messages"
        ),  # used to track the conversations
    ],
)

chain: LLMChain = LLMChain(
    llm=chat_llm,
    prompt=prompt,
    memory=memory,  # Add memory
)

In [21]:
content: str = "Hello, what is 20+5?"
result: dict[str, Any] = chain({"content": content})

# Follow up question
content: str = "subtract 10 from the result"
result: dict[str, Any] = chain({"content": content})


print(result)

{
    'content': 'subtract 10 from the result',
    'messages': [
        HumanMessage(content='Hello, what is 20+5?', additional_kwargs={}, example=False),
        AIMessage(content='20 + 5 equals 25.', additional_kwargs={}, example=False),
        HumanMessage(content='subtract 10 from the result', additional_kwargs={}, example=False),
        AIMessage(
            content='To subtract 10 from the result, you would subtract 10 from 25:\n\n25 - 10 = 15',
            additional_kwargs={},
            example=False
        ),
        HumanMessage(content='Hello, what is 20+5?', additional_kwargs={}, example=False),
        AIMessage(content='The result of 20 + 5 is 25.', additional_kwargs={}, example=False)
    ],
    'text': 'To subtract 10 from the result, you would subtract 10 from 25:\n\n25 - 10 = 15'
}

<hr><br>

## Loading Files With Document Loaders

[![image.png](https://i.postimg.cc/8CDcCFjL/image.png)](https://postimg.cc/TK7f73jP)

<br>

### Text Loader

[![image.png](https://i.postimg.cc/yNV6HwbP/image.png)](https://postimg.cc/rDP2GYc0)

In [22]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [23]:
fp: Path = Path("../../../data/facts.txt")
loader: TextLoader = TextLoader(file_path=fp)
docs: list[Any] = loader.load()

print(docs)

[
    Document(
        page_content='1. "Dreamt" is the only English word that ends with the letters "mt."\n2. An ostrich\'s eye 
is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of life.\n4. 
A snail can sleep for three years.\n5. The longest word in the English language is 
\'pneumonoultramicroscopicsilicovolcanoconiosis.\'\n6. The elephant is the only mammal that can\'t jump.\n7. The 
letter \'Q\' is the only letter not appearing in any U.S. state name.\n8. The heart of a shrimp is located in its 
head.\n9. Australia is the only continent covered by a single country.\n10. The Great Wall of China is 
approximately 13,171 miles long.\n11. Bananas are berries, but strawberries aren\'t.\n12. The Sphinx of Giza has 
the body of a lion and the head of a human.\n13. The first computer bug was an actual bug trapped in a 
computer.\n14. Neil Armstrong was the first man to walk on the moon.\n15. The Eiffel Tower in Paris leans slightly 
in the sun due to thermal expansion.\n16. Queen Elizabeth II is the longest-reigning current monarch.\n17. The 
Leaning Tower of Pisa took 200 years to construct.\n18. Angel Falls is the highest waterfall in the world, located 
in Venezuela.\n19. Sword swallowing is a skill that takes 3-10 years to learn.\n20. Isaac Newton invented the cat 
flap.\n21. Earth, Texas, is the only place on Earth named \'Earth.\'\n22. Thomas Edison, who invented the 
lightbulb, was afraid of the dark.\n23. The Pacific Ocean is the largest ocean on Earth, covering more than 60 
million square miles.\n24. Zeus was the king of the Greek gods according to ancient Greek myth.\n25. There are 
about 7,000 feathers on an eagle.\n26. Marie Curie was the first woman to win a Nobel Prize and remains the only 
person to have won in two different fields—Physics and Chemistry.\n27. The Sahara Desert is the largest hot desert 
in the world.\n28. There are 86,400 seconds in a day.\n29. Earth is the only known planet that supports life.\n30. 
The first Olympic Games were held in Ancient Greece in 776 B.C.\n31. The Mona Lisa has no visible eyebrows.\n32. A 
ball of glass will bounce higher than a ball of rubber.\n33. Penguins are the only bird that can jump.\n34. The 
shortest war in history was between Britain and Zanzibar on August 27, 1896. Zanzibar surrendered after 38 
minutes.\n35. Dolphins sleep with one eye open.  \n36. There are 1,792 steps to the top of the Eiffel Tower.\n37. 
Tug-of-war was an Olympic event between 1900 and 1920.\n38. The Nile River is the longest river in the world, 
measuring about 4,135 miles.  \n39. Lightning strikes the Earth 100 times every second.\n40. Charlie Chaplin once 
won third place in a Charlie Chaplin look-alike contest.\n41. Mount Everest is the tallest mountain in the 
world.\n42. No two zebras have the same pattern of stripes.\n43. The Great Barrier Reef is the world\'s largest 
coral reef system.\n44. Albert Einstein’s brain was stolen after his death.\n45. Martin Luther King Jr. delivered 
his famous "I Have A Dream" speech in 1963.\n46. Antarctica is the least populated continent on Earth.\n47. 
Chocolate was once used as a currency.\n48. Corn is on every continent except Antarctica.\n49. Mount Kilimanjaro is
the tallest free-standing mountain in the world (isn\'t part of a mountain range).\n50. Canada has more lakes than 
the rest of the world combined.\n51. 10% of the world\'s population is left-handed.\n52. The Statue of Liberty was 
gifted to the U.S. by France.\n53. The shortest war in history was between Britain and Zanzibar in 1896, lasting 38
minutes.\n54. The kangaroo and the emu are featured on the Australian coat of arms because neither animal can move 
backward, indicating progress.\n55. The bull shark, the hippopotamus, and the crocodile are the three animals 
responsible for the most human deaths in Africa.\n56. An apple, onion, and potato all have the same taste. The 
differences in flavor are caused by their smell.\n57. An

In [24]:
CHUNK_SIZE: int = 200
CHUNK_OVERlAP: int = 0

# Used to chunk the texts
text_splitter = CharacterTextSplitter(
    chunk_size=CHUNK_SIZE,  # 1. dets the rel. number of chars per chunk
    separator="\n",  # 2. each hunk is separated by this!
    chunk_overlap=CHUNK_OVERlAP,
)

# Load and split
docs: list[Any] = loader.load_and_split(text_splitter=text_splitter)

for d in docs:
    print(d.page_content)

1. "Dreamt" is the only English word that ends with the letters "mt."
2. An ostrich's eye is bigger than its brain.
3. Honey is the only natural food that is made without destroying any kind of life.

4. A snail can sleep for three years.
5. The longest word in the English language is 'pneumonoultramicroscopicsilicovolcanoconiosis.'
6. The elephant is the only mammal that can't jump.

7. The letter 'Q' is the only letter not appearing in any U.S. state name.
8. The heart of a shrimp is located in its head.
9. Australia is the only continent covered by a single country.

10. The Great Wall of China is approximately 13,171 miles long.
11. Bananas are berries, but strawberries aren't.
12. The Sphinx of Giza has the body of a lion and the head of a human.

13. The first computer bug was an actual bug trapped in a computer.
14. Neil Armstrong was the first man to walk on the moon.

...

109. Instead of bones, sharks have a skeleton made from cartilage.
110. On Mars, sunsets are blue.
111. The largest volcano in the solar system, Olympus Mons, is on Mars.

112. Saturn's density is low enough that the planet would float in water.
113. Starfish can regenerate their own arms.
114. French Fries originated in Belgium.

115. Cell phones have 18 times more bacteria than toilet handles.
116. Venus is the hottest planet in the solar system.
117. The Moon is drifting away from Earth every year.

118. The original Star-Spangled Banner was sewn in Baltimore.
119. The average adult spends more time on the toilet than they do exercising.

120. The world's quietest room is -9 decibels -- so quiet you can hear your blood flowing.

### Embeddings

- In the field of Natural Language Processing (NLP), embeddings are a powerful technique for representing words, phrases, or sentences as dense numerical vectors. 
- These vectors capture semantic and syntactic relationships between words, allowing machines to process and understand text in a more meaningful way.

[![image.png](https://i.postimg.cc/02BkcpJk/image.png)](https://postimg.cc/ppKMd5Z4)


In [25]:
from langchain.embeddings import OpenAIEmbeddings

In [26]:
emb_model = OpenAIEmbeddings()

emb_model.embed_query("It's good to be good")

[0.012109475210309029,
 -0.004843165632337332,
 0.012602847069501877,
 -0.04598977416753769,
 -0.029252594336867332,
 0.017324231564998627,
 -0.018560783937573433,
 -0.0182610135525465,
 -0.013439706526696682,
 -0.02647971920669079,
 0.02097143791615963,
 0.0024481250438839197,
 0.013577100820839405,
 0.0008399816579185426,
 -0.01267778966575861,
 -0.0029009031131863594,
 0.06260205060243607,
 -0.013164917007088661,
 0.00835609994828701,
 0.016237562522292137,
 -0.019435115158557892,
 -0.005617572460323572,
 -0.018573274835944176,
 -0.01779886707663536,
 -0.00858717318624258,
 0.0020905863493680954,
 -0.005258472636342049,
 -0.030351752415299416,
 0.018910516053438187,
 -0.015075952745974064,
 0.011191428638994694,
 0.004546517971903086,
 -0.015100933611392975,
 0.003890770021826029,
 -0.009255411103367805,
 -0.026329834014177322,
 -0.022495269775390625,
 -0.0037564977537840605,
 0.024531211704015732,
 0.012171926908195019,
 0.020446838811039925,
 -0.0050773611292243,
 0.00142781261820

### Vector Stores

- Qdrant

In [27]:
from langchain.vectorstores import Qdrant

In [28]:
url: str = "http://localhost:6333/"
qdrant = Qdrant.from_documents(
    docs,
    emb_model,
    url=url,
    prefer_grpc=False,
    collection_name="facts",
    force_recreate=False,
)

In [29]:
query: str = "What is an interesting fact about the English language"
found_docs = qdrant.similarity_search(query, k=2)

print(found_docs)

[
    Document(
        page_content='1. "Dreamt" is the only English word that ends with the letters "mt."\n2. An ostrich\'s eye 
is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of life.',
        metadata={'source': '../../../data/facts.txt'}
    ),
    Document(
        page_content='1. "Dreamt" is the only English word that ends with the letters "mt."\n2. An ostrich\'s eye 
is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of life.',
        metadata={'source': 'data/facts.txt'}
    )
]

In [30]:
print(found_docs[0].page_content)

1. "Dreamt" is the only English word that ends with the letters "mt."
2. An ostrich's eye is bigger than its brain.
3. Honey is the only natural food that is made without destroying any kind of life.

<br>

### Retrieval Chain

[![image.png](https://i.postimg.cc/X7SX9Rtj/image.png)](https://postimg.cc/qgwkT5kS)

In [31]:
from langchain.chains import RetrievalQA

In [32]:
# Retriever DB
retriever = qdrant.as_retriever()

chain = RetrievalQA.from_chain_type(
    llm=chat_llm,
    retriever=retriever,
    chain_type="stuff",  #  it feeds the LLM a prompt built from document list.
    return_source_documents=True,  # to easily detect hallucination
    input_key="query",
)

In [33]:
query: str = "What is an interesting fact about the English language"

# result: str = chain.run(query)
result: dict[str, Any] = chain({"query": query})
print(result)

{
    'query': 'What is an interesting fact about the English language',
    'result': 'An interesting fact about the English language is that "dreamt" is the only English word that ends 
with the letters "mt."',
    'source_documents': [
        Document(
            page_content='1. "Dreamt" is the only English word that ends with the letters "mt."\n2. An ostrich\'s 
eye is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of 
life.',
            metadata={'source': '../../../data/facts.txt'}
        ),
        Document(
            page_content='1. "Dreamt" is the only English word that ends with the letters "mt."\n2. An ostrich\'s 
eye is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of 
life.',
            metadata={'source': '../../../data/facts.txt'}
        ),
        Document(
            page_content='1. "Dreamt" is the only English word that ends with the letters "mt."\n2. An ostrich\'s 
eye is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of 
life.',
            metadata={'source': 'data/facts.txt'}
        ),
        Document(
            page_content="4. A snail can sleep for three years.\n5. The longest word in the English language is 
'pneumonoultramicroscopicsilicovolcanoconiosis.'\n6. The elephant is the only mammal that can't jump.",
            metadata={'source': '../../../data/facts.txt'}
        )
    ]
}

### Using The Debug Option

```python
import langchain

langchain.debug = True
```

In [34]:
import langchain

langchain.debug = True


query: str = "What is an interesting fact about the English language"

result: dict[str, Any] = chain({"query": query})
print(result)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is an interesting fact about the English language"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is an interesting fact about the English language",
  "context": "1. \"Dreamt\" is the only English word that ends with the letters \"mt.\"\n2. An ostrich's eye is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of life.\n\n1. \"Dreamt\" is the only English word that ends with the letters \"mt.\"\n2. An ostrich's eye is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of life.\n\n1. \"Dreamt\" is the only English word that ends with the letters \"mt.\"\n2. An ostrich's eye is bigger than its brain.\n3. Honey is

{
    'query': 'What is an interesting fact about the English language',
    'result': 'An interesting fact about the English language is that "dreamt" is the only English word that ends 
with the letters "mt."',
    'source_documents': [
        Document(
            page_content='1. "Dreamt" is the only English word that ends with the letters "mt."\n2. An ostrich\'s 
eye is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of 
life.',
            metadata={'source': '../../../data/facts.txt'}
        ),
        Document(
            page_content='1. "Dreamt" is the only English word that ends with the letters "mt."\n2. An ostrich\'s 
eye is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of 
life.',
            metadata={'source': '../../../data/facts.txt'}
        ),
        Document(
            page_content='1. "Dreamt" is the only English word that ends with the letters "mt."\n2. An ostrich\'s 
eye is bigger than its brain.\n3. Honey is the only natural food that is made without destroying any kind of 
life.',
            metadata={'source': 'data/facts.txt'}
        ),
        Document(
            page_content="4. A snail can sleep for three years.\n5. The longest word in the English language is 
'pneumonoultramicroscopicsilicovolcanoconiosis.'\n6. The elephant is the only mammal that can't jump.",
            metadata={'source': '../../../data/facts.txt'}
        )
    ]
}

In [35]:
type(qdrant)

langchain.vectorstores.qdrant.Qdrant